# Reporte de avances modelo Sentan-I

In [10]:
from Src.pre_processing import load_data, split, to_tensor, to_labels 
from Src.Sentan_Model import Sentan_dropout, Sentan_simple
from Src.Dias_Model import Dias_Model
from Src.pre_processing import split, to_tensor

import torch
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch import nn

from torchmetrics import F1Score, Recall, Precision, Accuracy
import wandb


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Ciclo de entrenamiento

In [11]:

def train(model, X_train, y_train, X_test, y_test, device, optimizer, lossFn, epochs):
    f1 = F1Score(num_classes=8)
    recall = Recall(average='macro', num_classes=8)
    precision = Precision(average='macro', num_classes=8)
    accuracy = Accuracy()

    X_train, y_train = to_tensor(X_train, y_train)
    X_test, y_test = to_tensor(X_test, y_test)
    X_train, y_train = X_train.to(device), y_train.to(device)
    X_test, y_test = X_test.to(device), y_test.to(device)

    # Entrenamiento del modelo
    model.train()
    for epoch in range(0, epochs):

        optimizer.zero_grad()

        pred = model(X_train)

        loss = lossFn(pred, y_train)
        loss.backward()
        optimizer.step()

        train_accuracy = accuracy(pred, y_train)

        # Validacion
        with torch.no_grad():
            pred = model(X_test)

            # Login de resultados a Weights and biases
            wandb.log({'Train Accuracy': train_accuracy, 'Train Loss': loss, 'Validation Accuracy': accuracy(
                pred, y_test), 'F1 Score': f1(pred, y_test), 'Recall': recall(pred, y_test), 'Precision': precision(pred, y_test)})


### Kfold Cross validation training

In [12]:
def train_kfold(model_builder, X, y, n_folds, optimizer_builder, lossFn, device='cpu', lr=1e-3, epochs=100):
    
    f1 = F1Score(num_classes=8)
    recall = Recall(average='macro', num_classes=8)
    precision = Precision(average='macro', num_classes=8)
    accuracy = Accuracy()

    # K-folds
    for train_idx, test_idx in split(X, y, splits=n_folds):


        # Activación del entorno
        wandb.init(project="Clasificador de sentimientos", entity="panas")

        model = model_builder().to(device)
        optimizer = optimizer_builder(
            model.parameters(), lr=1e-3, weight_decay=1e-5)

        # Fold Data
        x_train = X[train_idx]
        y_train = y[train_idx]

        x_test = X[test_idx]
        y_test = y[test_idx]

        x_train, y_train = to_tensor(x_train, y_train)
        x_test, y_test = to_tensor(x_test, y_test)

        # Train loop
        model.train()
        for epoch in range(0, epochs):

            # Training
            x_train = x_train.to(device)
            y_train = y_train.to(device)

            optimizer.zero_grad()

            pred = model(x_train)

            loss = lossFn(pred, y_train)
            loss.backward()
            optimizer.step()

            train_accuracy = accuracy(pred, y_train)

        # Validation
            with torch.no_grad():

                x_test = x_test.to(device)
                y_test = y_test.to(device)
                pred = model(x_test)

                wandb.log({'Train Accuracy': train_accuracy, 'Train Loss': loss, 'Validation Accuracy': accuracy(
                    pred, y_test), 'F1 Score': f1(pred, y_test), 'Recall': recall(pred, y_test), 'Precision': precision(pred, y_test)})


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/panas/Clasificador%20de%20sentimientos/12f4vxqb/file_stream
NoneType: None


### Preparación de los datos

Los datos serán cargados desde una versión del conjunto de datos RAVDESS, se representan como un conjunto de 193 variables por cada muestra.

In [13]:
data_path = 'Data/data.pkl'
X, y, labels = load_data(data_path)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Hiperparametros para pruebas

In [14]:
epochs = 10
learning_rate = 1e-3
batch_size = X_train.shape[0]


# Configuración de entorno weights and biases
wandb.config = {
    "model-name": 'Dias model',
    "learning_rate": learning_rate,
    "epochs": epochs,
    "batch_size": batch_size

}

### Entrenamiento simle

In [15]:
# Entrenamiento de n modelos
#models = [Dias_Model(), Dias_Model()]


# Model loop
#for model in models:
    
    # Activación del entorno
#    wandb.init(project="Clasificador de sentimientos", entity="panas")
#    model = model.to(device)


#    loss_fn = nn.CrossEntropyLoss()
#    optimizer = Adam(model.parameters(), lr=learning_rate)


    # Entrenamiento del modelo
#    train(model, X_train, y_train, X_test, y_test, device, optimizer, lossFn=loss_fn, epochs=epochs)

### Training kfold

In [16]:
# Entrenamiento de n modelos
models = [Dias_Model, Sentan_dropout, Sentan_simple]
folds = 10

# Model loop
for model in models:
    
    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam

    train_kfold(model, X, y, folds,  optimizer, loss_fn, device, learning_rate, epochs=epochs)

wandb: ERROR Error while calling W&B API: Error 1062: Duplicate entry '1102012-12f4vxqb' for key 'PRIMARY' (<Response [409]>)
Thread SenderThread:
Traceback (most recent call last):
  File "/home/solrak/Documents/clasificador_de_sentimientos/Prototypes/proto_env/lib/python3.10/site-packages/wandb/apis/normalize.py", line 22, in wrapper
    return func(*args, **kwargs)
  File "/home/solrak/Documents/clasificador_de_sentimientos/Prototypes/proto_env/lib/python3.10/site-packages/wandb/sdk/internal/internal_api.py", line 1319, in upsert_run
    response = self.gql(mutation, variable_values=variable_values, **kwargs)
  File "/home/solrak/Documents/clasificador_de_sentimientos/Prototypes/proto_env/lib/python3.10/site-packages/wandb/sdk/lib/retry.py", line 108, in __call__
    result = self._call_fn(*args, **kwargs)
  File "/home/solrak/Documents/clasificador_de_sentimientos/Prototypes/proto_env/lib/python3.10/site-packages/wandb/sdk/internal/internal_api.py", line 140, in execute
    return 

Problem at: /tmp/ipykernel_2691/2319210191.py 13 train_kfold


Traceback (most recent call last):
  File "/home/solrak/Documents/clasificador_de_sentimientos/Prototypes/proto_env/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 1876, in _atexit_cleanup
    self._on_finish()
  File "/home/solrak/Documents/clasificador_de_sentimientos/Prototypes/proto_env/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 1994, in _on_finish
    self._backend.interface.communicate_poll_exit()
  File "/home/solrak/Documents/clasificador_de_sentimientos/Prototypes/proto_env/lib/python3.10/site-packages/wandb/sdk/interface/interface.py", line 653, in communicate_poll_exit
    resp = self._communicate_poll_exit(poll_exit)
  File "/home/solrak/Documents/clasificador_de_sentimientos/Prototypes/proto_env/lib/python3.10/site-packages/wandb/sdk/interface/interface_shared.py", line 419, in _communicate_poll_exit
    result = self._communicate(rec)
  File "/home/solrak/Documents/clasificador_de_sentimientos/Prototypes/proto_env/lib/python3.10/site-packages/wa

Exception: problem